In [11]:
import os
import glob
import numpy as np
import polars as pl
from dotenv import load_dotenv

In [2]:
load_dotenv(r"C:\Users\by003457\workspace\perfectdays\.env")

True

In [3]:
NEWS_PARQUET_MONTH_DIR = os.environ["NEWS_PARQUET_MONTH_DIR"]

In [10]:
mmfiles = sorted(glob.glob(os.path.join(NEWS_PARQUET_MONTH_DIR, "*3pty_ko.npz")))
mmfiles[:5]

['L:\\MED\\TRAN\\2025_BankRegDataCollection\\rtrs_news\\monthly\\1999-03_embeddings_3pty_ko.npz',
 'L:\\MED\\TRAN\\2025_BankRegDataCollection\\rtrs_news\\monthly\\1999-04_embeddings_3pty_ko.npz',
 'L:\\MED\\TRAN\\2025_BankRegDataCollection\\rtrs_news\\monthly\\1999-05_embeddings_3pty_ko.npz',
 'L:\\MED\\TRAN\\2025_BankRegDataCollection\\rtrs_news\\monthly\\1999-06_embeddings_3pty_ko.npz',
 'L:\\MED\\TRAN\\2025_BankRegDataCollection\\rtrs_news\\monthly\\1999-07_embeddings_3pty_ko.npz']

In [12]:
mmfile = mmfiles[-1]

In [13]:
# read npz file
mm_data = np.load(mmfile)

In [14]:
mm_data.keys()

KeysView(NpzFile 'L:\\MED\\TRAN\\2025_BankRegDataCollection\\rtrs_news\\monthly\\2025-10_embeddings_3pty_ko.npz' with keys: ids, embeddings)

In [15]:
# print shape of each array
for key in mm_data.keys():
    print(f"{key}: {mm_data[key].shape}")   

ids: (173580,)
embeddings: (173580, 768)
